# Assignment Set 1
## 1.1 Vibrating string

The problem at hand is the numerical solution of a one-dimensional wave equation, 
which might describe the vibrations of a uniform string, the transport of voltage 
and current along a lossless transmission line, the pressure and flow rate of a 
compressible liquid or gas in a pipe, sound waves in gases or liquids, and optical waves. 
In this example, the domain of the wave equation is a string.

The one-dimensional wave equation is

$$
\frac{\partial^2 \Psi}{\partial t^2}
=
c^2 \frac{\partial^2 \Psi}{\partial x^2}
$$

The solution $\Psi(x,t)$ is the vibration amplitude expressed as a function 
of position and time. The problem becomes fully posed with the addition of 
boundary conditions on the spatial domain, and initial position and velocity 
distributions.

We attempt a numerical solution method by introducing a uniform discretization 
of the spatial and temporal domains, and approximating the partial differential 
equation by finite difference expressions. The grid points of this problem consist 
of discrete nodal points at which $\Psi$ is to be evaluated.

**A. (0.5 point)** Discretize the wave equation, and write it in a form suitable for implementing in a computer program. Assume that the boundaries are fixed, 
$\Psi(x=0,t)=0$, $\Psi(x=L,t)=0$. $L$ is the length of the string. Take $L=1$ for simplicity. Divide the string in $N$ intervals, so that the interval length is 
$\Delta x = L/N$. Also consider the boundary cases.

If you use Euler’s method, you need to use both $\Psi(x,t)$ and $\Psi_t(x,t)$ as variables. Or use the stepping method from the lectures, which uses $\Psi$ at the two most recent time points to calculate it at the next one.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import json

## 1.3 The Time Independent Diffusion Equation

Now assume that we are not very much interested in the time development of the 
concentration profile (i.e. the transient behavior), but only in the steady state. 
In that case it is possibly more effective to directly solve the time independent 
diffusion equation:

$$
\nabla^2 c = 0.
\tag{10}
$$

This is the famous Laplace equation. We again assume the same boundary 
conditions as in the previous section. Taking the same spatial discretization as before, 
and again applying the same 5-point stencil for the second order derivatives, 
eq.~(7) transforms to the following set of finite difference equations:

$$
\frac{1}{4}
\left(
c_{i+1,j}
+
c_{i-1,j}
+
c_{i,j+1}
+
c_{i,j-1}
\right)
=
c_{i,j}.
\tag{11}
$$

for all values of $(i,j)$. Note that the superscript $k$ is no longer present, 
because the time behavior is now suppressed. Many methods exist to solve the 
equations. We will here concentrate on iterative methods, because they are 
potentially efficient, and demand less memory than direct methods (and allow 
for relatively easy parallelism). A direct method is tried in exercise set 3.

## 1.4 The Jacobi Iteration

Using now the superscript $k$ to denote the $k$-th iteration, the Jacobi iteration is immediately suggested from eq.~(11):

$$
c_{i,j}^{k+1}
=
\frac{1}{4}
\left(
c_{i+1,j}^{k}
+
c_{i-1,j}^{k}
+
c_{i,j+1}^{k}
+
c_{i,j-1}^{k}
\right).
\tag{12}
$$

Note that eq.~(12) is easily derived from eq.~(7) by putting

$$
\frac{\delta t D}{\delta x^2} = \frac{1}{4}.
\tag{13}
$$

i.e. the Jacobi iteration is nothing but the solution of the time-dependent equation using the scheme from eq.~(7) with the maximum allowed time step. This suggests that more efficient iterative methods are needed.

There is one noticeable difference between the Jacobi iteration and the solution of the time dependent equation. In the time dependent case one defines the time step and the total time that should be simulated. In an iterative method one needs a stopping condition. This stopping condition typically is some global measure.

Assume that the stopping condition is such that the solution is assumed to be converged if for all values of $(i,j)$

$$
\delta
\equiv
\max_{i,j}
\left|
c_{i,j}^{k+1}
-
c_{i,j}^{k}
\right|
<
\varepsilon,
\tag{14}
$$

where $\varepsilon$ is some small number, say $10^{-5}$.

For implementing the Jacobi iteration, note that separate matrices are needed for $c_{i,j}^{k}$ and $c_{i,j}^{k+1}$; one cannot simply use one matrix and update it, as one would then overwrite values that are needed later.

In [10]:
from utils import jacobi
u = jacobi(50,50)


In [11]:
u

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 4.99315875e-01, 6.96284237e-01, ...,
        6.96284237e-01, 4.99315875e-01, 0.00000000e+00],
       [0.00000000e+00, 3.00981331e-01, 4.97267643e-01, ...,
        4.97267643e-01, 3.00981331e-01, 0.00000000e+00],
       ...,
       [0.00000000e+00, 5.20011301e-04, 1.03771178e-03, ...,
        1.03771178e-03, 5.20011301e-04, 0.00000000e+00],
       [0.00000000e+00, 2.58714118e-04, 5.16279475e-04, ...,
        5.16279475e-04, 2.58714118e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]], shape=(50, 50))

## 1.5 The Gauss–Seidel Iteration

An improvement over the Jacobi iteration is the Gauss–Seidel iteration, where during the iteration a new value is used as soon as it has been calculated. Assuming that the iteration proceeds along the rows (i.e. incrementing $i$ for fixed $j$), the Gauss–Seidel iteration reads

$$
c_{i,j}^{k+1}
=
\frac{1}{4}
\left(
c_{i+1,j}^{k}
+
c_{i-1,j}^{k+1}
+
c_{i,j+1}^{k}
+
c_{i,j-1}^{k+1}
\right).
$$

The Gauss–Seidel iteration is not a big improvement over the Jacobi iteration (in terms of the amount of iterations needed for convergence) and is only a first step in introducing the Successive Over Relaxation method (next section). However, the update can be performed *in place*.

## 1.6 Successive Over Relaxation

Now that you have the parallel Gauss–Seidel iteration in place, it is easy to take the next and final step. The Gauss–Seidel iteration did not provide a huge improvement over the Jacobi iteration. A next improvement comes from the Successive Over Relaxation (SOR). SOR is obtained from Gauss–Seidel by an over-correction of the new iterate, in formula

$$
c_{i,j}^{k+1}
=
\frac{\omega}{4}
\left(
c_{i+1,j}^{k}
+
c_{i-1,j}^{k+1}
+
c_{i,j+1}^{k}
+
c_{i,j-1}^{k+1}
\right)
+
(1-\omega)c_{i,j}^{k}.
$$

This method converges only for $0 < \omega < 2$. For $\omega < 1$ the method is called under-relaxation. The new value is then the weighted average of the Gauss–Seidel method and the previous value. For $\omega = 1$ we recover the Gauss–Seidel iteration.

It turns out that for our diffusion problem the optimal $\omega$ (that minimizes the number of iterations) lies somewhere in the interval $1.7 < \omega < 2$. The exact value depends on the grid size $N$.